In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [2]:
train_dir = 'C:/Documents/Mohamed Elsayed CV/Computer-Vision-Tasks-main/Alzheimer_s Dataset/test'  
test_dir = 'C:/Documents/Mohamed Elsayed CV/Computer-Vision-Tasks-main/Alzheimer_s Dataset/test'

In [3]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [4]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [5]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [6]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [7]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step


In [9]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step


In [10]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3225701	total: 200ms	remaining: 3m 20s
1:	learn: 1.2635601	total: 240ms	remaining: 1m 59s
2:	learn: 1.2186601	total: 285ms	remaining: 1m 34s
3:	learn: 1.1807625	total: 322ms	remaining: 1m 20s
4:	learn: 1.1473287	total: 361ms	remaining: 1m 11s
5:	learn: 1.1165962	total: 399ms	remaining: 1m 6s
6:	learn: 1.0921009	total: 441ms	remaining: 1m 2s
7:	learn: 1.0680591	total: 481ms	remaining: 59.6s
8:	learn: 1.0475403	total: 523ms	remaining: 57.5s
9:	learn: 1.0290488	total: 563ms	remaining: 55.8s
10:	learn: 1.0131934	total: 603ms	remaining: 54.2s
11:	learn: 0.9984309	total: 640ms	remaining: 52.7s
12:	learn: 0.9848170	total: 679ms	remaining: 51.5s
13:	learn: 0.9729003	total: 718ms	remaining: 50.6s
14:	learn: 0.9601245	total: 756ms	remaining: 49.7s
15:	learn: 0.9484325	total: 796ms	remaining: 48.9s
16:	learn: 0.9380423	total: 833ms	remaining: 48.1s
17:	learn: 0.9290448	total: 872ms	remaining: 47.6s
18:	learn: 0.9204869	total: 912ms	remaining: 47.1s
19:	learn: 0.9134639	total: 953ms	re

In [12]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [13]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 56.29%


In [14]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [15]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [ ]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('C:/Users/ASUS/Downloads/Mohamed Elsayed CV/Computer-Vision-Tasks-main/Task4/cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('C:/Users/ASUS/Downloads/Mohamed Elsayed CV/Computer-Vision-Tasks-main/Task4/catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = "C:/Documents/Mohamed Elsayed CV/Computer-Vision-Tasks-main/Alzheimer_s Dataset/test/NonDemented/26 (88).jpg"
result = classify_image(img_path)
print(f"The image is classified as: {result}")
